Con este programa calculamos percentiles por cada quality model y clasificamos las calidades

In [8]:
import os

home = os.path.expanduser("~")

db_name=  'test_1' #'test_new_creators_4'
input_dir = os.path.join(home, 'video_resources', 'image_datasets')
output_dir = os.path.join(home, 'video_resources', 'sql_database')
database_names=['enfsi','lfw','forenface'] # 'lfw', 'scface',  'enfsi', 'enfsi2015', 'xqlfw', 'utkface','chokepoint','forenface'
detector_names=['mtcnn'] # 'dlib', 'mtcnn', 'mtcnn_serfiq'
embedding_model_names=["QMagFace"] # , "Dlib", "ArcFace",
attributes_to_update=[]  # 'gender', 'age', 'emotion', 'race'
quality_model_names = ['ser_fiq', 'tface'] # 'confusion_score', 'ser_fiq',

In [9]:
from sql_face.sqldb import SQLDataBase

database = SQLDataBase(db_name=db_name,
                        input_dir = input_dir,
                        output_dir_name = output_dir,
                        database_names = database_names,
                        detector_names = detector_names,
                        embedding_model_names = embedding_model_names,
                        quality_model_names = quality_model_names)

In [10]:


import pandas as pd
import numpy as np
from tqdm import tqdm
from sql_face.tables import EnfsiPair, EnfsiPair2015, QualityImage, FaceImage, CroppedImage, Image, QualityModel, QualityGroup

session = database.session

for qm in quality_model_names:

    sql = session.query(QualityImage).join(QualityModel).filter(QualityModel.name == qm).all()

    quality = [row.quality for row in sql if row.quality]



    percentiles = [20, 40, 60, 80, 99]
    percentiles[-1] = np.inf

    quaperc = np.percentile(quality,percentiles)

    n1 = 100
    n = n1
    for row in tqdm(sql):

        quality_group = next(X_Enum   for X_Enum, umbral in zip(QualityGroup,quaperc) if row.quality < umbral)
        row.quality_group = quality_group    
    
        if not n:
            session.commit()
            n = n1
        else:
            n-=1
    session.commit() 

  0%|          | 149/863754 [00:05<9:27:47, 25.35it/s] 


StopIteration: 

In [ ]:
# query = (session.query(EnfsiPair).limit(100).all())

# for img in query:
#     filter = ['image_quality']
#     cat = img.get_category(im_category_list = filter,fi_cat_list=[],detector='mtcnn',embedding_model='Arcface')

#     cat2 = img.get_category([],[],'mtcnn','Arcface')

#     a = 5
